<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B210516%5DTrain_and_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.utils.data
import torch.nn as nn
from datetime import datetime
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")

import random
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
# np.seterr(divide='ignore', invalid='ignore')
import itertools
import re
from time import sleep

In [ ]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k, use_cuda):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        if use_cuda==True:
            self.v = nn.Parameter(torch.ones(1, n_vis).cuda())
            self.h = nn.Parameter(torch.zeros(1, n_hid).cuda())
            self.W = nn.Parameter(torch.randn(n_hid, n_vis).cuda())
            self.k = k
        else:
            self.v = nn.Parameter(torch.ones(1, n_vis))
            self.h = nn.Parameter(torch.zeros(1, n_hid))
            self.W = nn.Parameter(torch.randn(n_hid, n_vis))
            self.k = k     

    def visible_to_hidden(self, v, beta):
        return torch.sigmoid(F.linear(v, self.W, self.h)*beta)

    def hidden_to_visible(self, h, beta):
        return torch.sigmoid(F.linear(h, self.W.t(), self.v)*beta)

    def free_energy(self, v):
        v_term = torch.matmul(v, self.v.t())
        w_x_h = F.linear(v, self.W, self.h)
        h_term = torch.sum(F.softplus(w_x_h), dim=1)
        return torch.mean(-h_term - v_term)
    
    def energy(self, v):
        v=v.bernoulli()
        h=torch.sigmoid(F.linear(v, self.W, self.h))
        h=h.bernoulli()
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())
    
    def energy2(self, v, h):
        v=v.bernoulli()
        h=h.bernoulli()
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())

    def forward(self, v):
        h = self.visible_to_hidden(v,1)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibb = self.hidden_to_visible(h,1).to(device)
            v_gibb = v_gibb.bernoulli()
            h = self.visible_to_hidden(v_gibb, 1).to(device)
            h = h.bernoulli()
        return v, v_gibb

In [ ]:

def decimal_to_binary(integer):
    string=bin(integer)[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def binary_to_decimal(list0):
    value=0
    for i in range(len(list0)):
        value+=list0[-i-1]*2**(i)
    return value


In [ ]:
# Complete infinite and zero T

# fullconfigs=[]
# for i in range(2**16):
#     fullconfigs.append(decimal_to_binary(i).detach().numpy())
# with open('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_inf_full.pkl', 'wb') as f:
#     pkl.dump(fullconfigs, f)
# random.shuffle(fullconfigs)
# with open('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_inf_sub.pkl', 'wb') as f:
#     pkl.dump(fullconfigs[:6000], f)


# fullconfigs=[]
# for i in range(int(2**16/2)-16):
#     fullconfigs.append(decimal_to_binary(0).detach().numpy())
#     fullconfigs.append(decimal_to_binary(2**16-1).detach().numpy())
# for i in range(16):
#     fullconfigs.append(decimal_to_binary(2**i).detach().numpy())
#     fullconfigs.append(decimal_to_binary(2**16-1-2**i).detach().numpy())
# subconfigs=[]
# for i in range(2984):
#     subconfigs.append(decimal_to_binary(0).detach().numpy())
#     subconfigs.append(decimal_to_binary(2**16-1).detach().numpy())
# for i in range(16):
#     subconfigs.append(decimal_to_binary(2**i).detach().numpy())
#     subconfigs.append(decimal_to_binary(2**16-1-2**i).detach().numpy())
# random.shuffle(fullconfigs)
# random.shuffle(subconfigs)
# with open('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_zero_full.pkl', 'wb') as f:
#     pkl.dump(fullconfigs, f)
# with open('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_zero_sub.pkl', 'wb') as f:
#     pkl.dump(subconfigs, f)

In [ ]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available()

True

In [124]:
n_hid_range=[1,2,4,8,16]
# n_hid_range=[2, 128, 24, 48, 96]
# n_hid_range=[128]


n_vis=9
k=5
n_epochs=10000
batch_size=512
lr=0.00001

In [125]:
from torch.utils.data import Dataset

class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

In [126]:
fullconfigs=pd.read_pickle('drive/MyDrive/loss_IG/3*3/3*3_cri_full.pkl')
# subconfigs=pd.read_pickle('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_cri_sub.pkl')

In [127]:
fulldata=CustomDataset(fullconfigs)
full_dataset = fulldata
full_loader = torch.utils.data.DataLoader(full_dataset, batch_size)

# subdata=CustomDataset(subconfigs)
# sub_dataset = subdata
# sub_loader = torch.utils.data.DataLoader(sub_dataset, batch_size)


In [130]:
def train_and_get_data(n_hid, model, lr, train_loader, por, T):
    if model==0:
        rbm = RBM(n_vis, n_hid, k, use_cuda=CUDA)
    else:
        rbm=model
    train_loss_list=[]
    train_op = optim.Adam(rbm.parameters(), lr)
    rbm.train()
    for epoch in range(n_epochs):
        train_loss_epoch = []
        for _, (data) in enumerate(train_loader):
            data=data.to(device)
            v, v_gibbs = rbm(data.view(-1, n_vis))
            train_loss = rbm.free_energy(v) - rbm.free_energy(v_gibbs)
            train_loss_epoch.append(train_loss.item())
            train_op.zero_grad()
            train_loss.backward()
            train_op.step()
        val=np.mean(train_loss_epoch)
        train_loss_list.append(val)
        if epoch>10 and train_loss_list[-1]*train_loss_list[-2]<0:
            break
    with open('drive/MyDrive/loss_IG/3*3/loss/2021-05-02_loss_n_hid={n_hid}_{por}_T={T}.pkl'.format(n_hid=n_hid, por=por, T=T), 'wb') as f:
        pkl.dump(train_loss_list, f)
    torch.save(rbm.state_dict(), 'drive/MyDrive/loss_IG/3*3/state_dict/2021-05-02_n_hid={n_hid}_{por}_T={T}'.format(n_hid=n_hid, por=por, T=T))
    

In [ ]:
for n_hid in [2,4,6,8]:
    for j in range(10):
        train_and_get_data(n_hid
#                    ,model=torch.load('ERBM,SGD/lr=0.01/models/2021-01-28_n_hid={n_hid}_vol=600_epoch=5000'.format(n_hid=n_hid)
                    ,0
                    ,lr
                    ,train_loader=full_loader
                    ,por='full'
                    ,T='cri{j}'.format(j=j+1))